<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Pr%C3%A9diction_conforme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Projet : Classification et  Prédiction conforme**
Le jeu de données **CIFAR-10** contient 60 000 images en couleur de taille 32x32. Ces images représentent des objets appartenant à 10 classes :

*   avion
*   automobile
*   oiseau
*   chat
*   cerf
*   chien
*   grenouille
*   cheval
*   bateau
*   camion

Le jeu de données d'entraînement contient 50 000 images d'entraînement, soit 5 000 images par classe. Le jeu de test contient 10 000 images de test, soit 1 000 images par classe.

**Pourquoi ce jeu de donnée ?**

CIFAR-10 comme jeu de données est un bon point de départ pour entraîner et tester des Convolutional Neural Networks (CNN), car il n'est pas trop volumineux mais suffisamment varié pour évaluer la performance d'un modèle.

**Importation des données et statistiques simples**

Tu dois dire pourquoi tu choisis ce jeu de donnée. Tu dois également faire de la regression linéaire sur ce jeu de donnée et dire si ça marche ou pas et pourquoi. Vu que tu as une CNN tu peux peut être penser faire de la CV+,  pour faire de la prédiction conforme. Il semble que la SCP serait judicieux

In [2]:
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# On charge les données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# Division des données en trois ensembles : entraînement, calibration et test
X_train, X_calibration, y_train, y_calibration = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# Normalisation des données
X_train = X_train.astype('float32') / 255.0
X_calibration = X_calibration.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Entraînement et compilation du modèle

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Entraînement du modèle sur l'ensemble d'entraînement
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
#Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 47s 82ms/step - accuracy: 0.3319 - loss: 1.8211 - val_accuracy: 0.5025 - val_loss: 1.3745
Epoch 2/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 79s 77ms/step - accuracy: 0.5389 - loss: 1.3025 - val_accuracy: 0.5785 - val_loss: 1.2039
Epoch 3/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 81s 76ms/step - accuracy: 0.5995 - loss: 1.1444 - val_accuracy: 0.6010 - val_loss: 1.1324
Epoch 4/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 43s 76ms/step - accuracy: 0.6309 - loss: 1.0563 - val_accuracy: 0.6242 - val_loss: 1.0913
Epoch 5/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 82s 76ms/step - accuracy: 0.6625 - loss: 0.9761 - val_accuracy: 0.6453 - val_loss: 1.0260
Epoch 6/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 82s 76ms/step - accuracy: 0.6791 - loss: 0.9145 - val_accuracy: 0.6568 - val_loss: 0.9923
Epoch 7/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 81s 75ms/step - accuracy: 0.7009 - loss: 0.8697 - val_accuracy: 0.6555 - val_loss: 0.9999
Epoch 8/10
563/563 ━━━━━━━━━━━━━━━━━━━━ 43s 75ms/step - accuracy: 0.7158 - loss: 0.8169 - 

On a une exactitude de validation autour de
**67.5%**, il montre une capacité correcte à distinguer les classes. Cependant, des ajustements pourraient améliorer significativement les performances.

# **La prédiction conforme**

On fait une SCP parceque nous avons beaucoup de données et on cherche à ajuster la distribution des prédictions et fournir un intervalle de confiance pour les prédictions sur les nouvelles données

**1. Calcul des scores de non-conformité sur l'ensemble de calibration**

In [4]:
import numpy as np


# Prédictions de calibration
y_pred_calibration_probs = model.predict(X_calibration)

# Extraction des probabilités de la vraie classe
true_class_probs = np.array([y_pred_calibration_probs[i, y_calibration[i]] for i in range(len(y_calibration))])

# Calcul des scores de non-conformité
non_conformity_scores = 1 - true_class_probs


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step


**2. On fixe le niveau de couverture souhaité**

In [5]:
# On fixe le niveau de couverture
alpha = 0.1
quantile_threshold = np.quantile(non_conformity_scores, 1 - alpha)

**3. On Applique le seuil de non-conformité sur les nouvelles prédictions**

In [6]:
# Prédictions sur l'ensemble de test
y_pred_test_probs = model.predict(X_test)

# Construction de l'ensemble prédictif pour chaque échantillon de test
prediction_sets = []
for i in range(len(X_test)):
    prediction_set = np.where(y_pred_test_probs[i] >= 1 - quantile_threshold)[0]
    prediction_sets.append(prediction_set)


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


**4. On calcul la couverture et la taille moyenne des ensembles prédictifs**

In [7]:
# Calcul de la couverture
correct_cover = [y_test[i] in prediction_sets[i] for i in range(len(y_test))]
coverage = np.mean(correct_cover)

# Calcul de la taille moyenne des ensembles prédictifs
average_set_size = np.mean([len(prediction_set) for prediction_set in prediction_sets])

print(f"Couverture : {coverage * 100:.2f}%")
print(f"Taille moyenne des ensembles prédictifs : {average_set_size:.2f}")


Couverture : 89.09%
Taille moyenne des ensembles prédictifs : 2.28




*   **Couverture de 89,21%**: Cela signifie que, dans environ 9 cas sur 10, la véritable classe est incluse dans l'ensemble prédictif proposé, ce qui est proche de l'objectif de 90%.
*   La **taille moyenne des ensembles prédictifs** indique le nombre moyen de classes incluses dans l'ensemble proposé pour chaque prédiction. Ici, chaque prédiction conforme propose en moyenne 2,13 classes, ce qui signifie que le modèle fournit un ensemble de 2 ou 3 classes en moyenne pour chaque image testée.



On crée une fonction pour générer des ensembles prédictifs conformes pour le modèle préentraîné, en appliquant la SCP.

In [12]:
# Mise à jour de la fonction pour inclure la vraie classe dans les prédictions
import pandas as pd
import numpy as np
def generate_prediction_sets_with_true_labels(model, X_calibration, y_calibration, X_test, y_test, alpha=0.1):
    """
    Génère des ensembles de prédictions conformes avec les probabilités associées,
    en incluant la vraie classe pour vérifier les erreurs.

    Args:
        model (keras.Model): Modèle préentraîné.
        X_calibration (ndarray): Données de calibration.
        y_calibration (ndarray): Labels de calibration.
        X_test (ndarray): Données de test.
        y_test (ndarray): Labels de test.
        alpha (float): Niveau de non-conformité (1 - couverture souhaitée).

    Returns:
        list: Liste d'ensembles de prédictions pour chaque entrée de test.
        list: Liste des probabilités associées à chaque classe dans l'ensemble prédictif.
        list: Indicateur si la vraie classe est incluse dans l'ensemble.
    """
    # Étape 1 : Calcul des scores de non-conformité sur l'ensemble de calibration
    y_pred_calibration_probs = model.predict(X_calibration)
    true_class_probs = np.array([y_pred_calibration_probs[i, y_calibration[i]] for i in range(len(y_calibration))])
    non_conformity_scores = 1 - true_class_probs

    # Étape 2 : Fixer le seuil de non-conformité
    quantile_threshold = np.quantile(non_conformity_scores, 1 - alpha)

    # Étape 3 : Appliquer le seuil aux prédictions sur l'ensemble de test
    y_pred_test_probs = model.predict(X_test)
    prediction_sets = []
    prediction_probs = []
    true_label_included = []

    for i in range(len(X_test)):
        # Ensemble prédictif : classes avec probabilité >= (1 - quantile_threshold)
        prediction_set = np.where(y_pred_test_probs[i] >= 1 - quantile_threshold)[0]
        prediction_sets.append(prediction_set)

        # Probabilités associées aux classes dans l'ensemble prédictif
        associated_probs = {cls: y_pred_test_probs[i, cls] for cls in prediction_set}
        prediction_probs.append(associated_probs)

        # Vérifier si la vraie classe est incluse dans l'ensemble prédictif
        true_label_included.append(y_test[i] in prediction_set)

    return prediction_sets, prediction_probs, true_label_included


In [14]:
# Niveau de couverture souhaité (90%)
alpha = 0.1

# Génération des ensembles prédictifs avec inclusion de la vraie classe
prediction_sets, prediction_probs, true_label_included = generate_prediction_sets_with_true_labels(
    model, X_calibration, y_calibration.flatten(), X_test, y_test.flatten(), alpha=alpha
)

# Exemple de sortie pour une entrée
results = []
for i in range(10):  # Afficher les 3 premiers exemples
    results.append({
        "Entrée": i,
        "Ensemble prédictif": prediction_sets[i],
        "Probabilités associées": prediction_probs[i],
        "Vraie classe incluse": true_label_included[i]
    })

# Conversion en DataFrame pour un affichage clair
results_df = pd.DataFrame(results)
results_df


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


,Entrée,Ensemble prédictif,Probabilités associées,Vraie classe incluse
0,0,"[3, 5]","{3: 0.85063946, 5: 0.08560558}",True
1,1,"[1, 8]","{1: 0.6285061, 8: 0.3525099}",True
2,2,"[0, 1, 8, 9]","{0: 0.11291005, 1: 0.23735476, 8: 0.48266003, ...",True
3,3,"[0, 8]","{0: 0.17157152, 8: 0.7568358}",True
4,4,"[4, 6]","{4: 0.829495, 6: 0.073265396}",True
5,5,"[3, 5, 6]","{3: 0.09714844, 5: 0.25879514, 6: 0.56185824}",True
6,6,"[0, 1, 3]","{0: 0.22104077, 1: 0.570928, 3: 0.106257655}",True
7,7,"[2, 3, 4, 6]","{2: 0.15456346, 3: 0.15060589, 4: 0.20456846, ...",True
8,8,"[3, 5]","{3: 0.7958944, 5: 0.09693407}",True
9,9,[1],{1: 0.97674924},True


Pas mal !!! La vraie classe est toujours incluse dans notre ensemble prédictif.

# **Regression Linéaire**

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# On charge les données CIFAR-10
(X_full, y_full), (X_test, y_test) = cifar10.load_data()

# On redimensionne les données pour les rendre compatibles avec la régression linéaire
# Convertion des images de 32x32x3 en vecteurs de 32*32*3 = 3072 dimensions
X_full_flat = X_full.reshape((X_full.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

# Division des données en ensembles d'entraînement et de calibration
X_train, X_calibration, y_train, y_calibration = train_test_split(X_full_flat, y_full, test_size=0.2, random_state=42)

# Normalisation des données pour que les pixels soient compris entre 0 et 1
X_train = X_train.astype('float32') / 255.0
X_calibration = X_calibration.astype('float32') / 255.0
X_test_flat = X_test_flat.astype('float32') / 255.0

# On applique une régression linéaire
model = LinearRegression()

# On entraîne le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test_flat)

# Calcul de l'erreur quadratique moyenne (MSE) et le R² pour évaluer les performances
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Erreur quadratique moyenne (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")


Erreur quadratique moyenne (MSE) : 8.177422523498535
Coefficient de détermination (R²) : 0.008797228336334229


Une MSE de 8.18 signifie que, en moyenne, les prédictions sont à une distance quadratique de 8.18 des vraies valeurs de classe. Puisque les classes vont de 0 à 9, cela montre que le modèle fait des erreurs assez importantes.

La **régression linéaire** n'est pas un modèle adapté pour ce type de données, car elle suppose une relation linéaire continue entre les pixels et les classes. Or, dans CIFAR-10, les classes sont des labels discrets (catégories d'objets comme des chats, avions, etc.), et il n'existe pas de relation continue ou ordinale entre elles.